In [1]:
# import libraries

import sys
import warnings
import os
os.environ['USE_PYGEOS'] = '0'

import geopandas as gpd
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

sys.path.append('../../utils')
import data_paths
from gridding import GriddingEngine

warnings.filterwarnings("ignore")

# Reload local modules on changes
%reload_ext autoreload
%autoreload 2

In [2]:

filename = 'linesource_Munich_2019_los_specific.gpkg'
gdf = gpd.read_file(data_paths.INVENTORY_PATH+filename)

In [13]:
species = 'NOx'

gdf['road_length'] = gdf.geometry.length * 1e-3  # convert to km
gdf[f'{species}_total'] = (gdf[f'PC_{species}'] + gdf[f'LCV_{species}'] + \
    gdf[f'HGV_{species}'] + gdf[f'BUS_{species}'] + gdf[f'MOT_{species}'])
gdf[f'{species}_total'] = gdf[f'{species}_total'] * gdf.geometry.length * 1e-9 # convert to t /km

print(f'Total {species}_total:', gdf[f'{species}_total'].sum())
print('\nShare by road type:')
print(gdf.groupby('road_type')[f'{species}_total'].sum() / gdf[f'{species}_total'].sum() * 100)

Total NOx_total: 3333.482648777689

Share by road type:
road_type
Access-residential            10.650588
Distributor/Secondary         40.466758
Motorway-Nat                  35.645574
TrunkRoad/Primary-City        13.226298
TrunkRoad/Primary-National     0.010783
Name: NOx_total, dtype: float64


In [14]:
path = '/Users/daniel_tum/Documents/code/drive-inventory/data/restricted_input/counting_data/counting_data_combined_until2024.parquet'

df_counts = pd.read_parquet(path)

## Number of individual counters per road type

In [30]:
df_counts[(df_counts['valid'] == True)].groupby('scaling_road_type')['road_link_id'].nunique()

scaling_road_type
Distributor/Secondary     64
Motorway-Nat              65
TrunkRoad/Primary-City    17
Name: road_link_id, dtype: int64

In [27]:
df_counts.head()

,road_link_id,vehicle_class,date,road_type,complete,valid,completness,scaling_road_type,sqv,day_type,...,14,15,16,17,18,19,20,21,22,23
0,38.0,BUS,2019-01-08,TrunkRoad/Primary-City,True,True,0.934041,TrunkRoad/Primary-City,0.0,0,...,10.0,10.0,5.0,5.0,3.0,4.0,4.0,1.0,6.0,4.0
1,38.0,BUS,2019-01-09,TrunkRoad/Primary-City,True,True,0.934041,TrunkRoad/Primary-City,0.0,0,...,9.0,11.0,5.0,10.0,6.0,9.0,2.0,2.0,4.0,5.0
2,38.0,BUS,2019-01-10,TrunkRoad/Primary-City,True,True,0.934041,TrunkRoad/Primary-City,0.0,0,...,16.0,9.0,7.0,7.0,2.0,1.0,10.0,3.0,1.0,5.0
3,38.0,BUS,2019-01-15,TrunkRoad/Primary-City,True,True,0.934041,TrunkRoad/Primary-City,0.0,0,...,23.0,17.0,35.0,20.0,18.0,24.0,6.0,2.0,1.0,8.0
4,38.0,BUS,2019-01-16,TrunkRoad/Primary-City,True,True,0.934041,TrunkRoad/Primary-City,0.0,0,...,27.0,32.0,25.0,33.0,21.0,25.0,6.0,1.0,3.0,12.0


## Eval number of days with imputation

In [54]:
df_med = df_counts[(df_counts['complete']) & (df_counts['valid'])].groupby(['vehicle_class',
                                                                   'scaling_road_type',
                                                                   'date'])['daily_value'].median()

df_med = pd.DataFrame(df_med).reset_index()
df_med['year'] = pd.DatetimeIndex(df_med['date']).year
#df_med.set_index(['vehicle_class', 'year', 'scaling_road_type'], inplace=True)

In [100]:

total_days = len(pd.date_range(start = f'{2020}-01-01', end = f'{2020}-12-31', freq = 'D'))
min_day = (df_med[df_med['year'].isin([2020])].groupby(['vehicle_class', 'scaling_road_type'])['date'].nunique())

share = min_day - total_days


pd.DataFrame(share).reset_index().pivot(index='scaling_road_type', columns='vehicle_class')



date                    
vehicle_class           BUS HGV LCV MOT  PC SUM
scaling_road_type                              
Distributor/Secondary     0   0   0   0   0   0
Motorway-Nat              0  -2   0   0  -3  -4
TrunkRoad/Primary-City   -6  -8 -15  -7 -37 -36

In [98]:
total_days

1461